# Elasto-plastic isotropic hardening model

Verification of the Legendre transform

In [ ]:
%matplotlib widget
from bmcs_matmod.api import GSMRM
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from bmcs_matmod.gsm.potentials.potential1d_t_e_vp_lih import Potential1D_T_E_VP_LIH_SymbExpr 
sp.init_printing()

## Direct solution of consistency condition

### Helmholtz free energy

In [ ]:
p1d = Potential1D_T_E_VP_LIH_SymbExpr()
p1d.F_

### Gibbs free energy

In [ ]:
eps_a = p1d.eps_a
sig_a = p1d.sig_a
dF_du = p1d.F_.diff(eps_a)
u_sig_ = sp.Matrix([ sp.solve(sp.Eq(sig_i, dF_du_i), u_i)[0] for sig_i, u_i, dF_du_i in 
                            zip(sig_a, eps_a, dF_du)])
subs_u_sig_ = dict(zip(eps_a, u_sig_))
sp.simplify(dF_du), subs_u_sig_

In [ ]:
sig_x_eps_ = (sig_a.T * eps_a)[0]
G_expr = p1d.F_ - sig_x_eps_
G_ = sp.simplify(G_expr.subs(subs_u_sig_))
G_

In [ ]:
G_sig = sp.simplify(p1d.F_.subs(subs_u_sig_).args[0] - sig_x_eps_.subs(subs_u_sig_))
G_sig

In [ ]:
sp.simplify((G_sig +  + p1d.sig * p1d.eps).subs(p1d.sig, p1d.E*(p1d.eps - p1d.eps_p)))

### GSM drivers

In [ ]:
gsm_F = GSMRM(
    name = 'gsm_F_1d_t_e_vp_lkh',
    u_vars = p1d.eps_a,
    sig_vars = p1d.sig_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = p1d.Sig_signs,
    F_expr = p1d.F_,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_,
    m_param_codenames =  p1d.m_param_codenames
)

gsm_G = GSMRM(
    name = 'gsm_G_1d_t_e_vp_lkh',
    u_vars = p1d.sig_a,
    sig_vars = p1d.eps_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_signs = p1d.Sig_signs,
    Sig_vars = p1d.Sig_vars,
    F_expr = G_,
    sig_sign = sp.Rational(-1),
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_,
    m_param_codenames =  p1d.m_param_codenames
)

In [ ]:
gsm_F.Sig_, gsm_G.Sig_

In [ ]:
(gsm_F.df_dSig_.T * gsm_F.dSig_dEps_ * gsm_F.Phi_, 
 gsm_G.df_dSig_.T * gsm_G.dSig_dEps_ * gsm_G.Phi_)

In [ ]:
gsm_F.df_dlambda_, gsm_G.df_dlambda_

In [ ]:
lambda_solved_F = gsm_F.f_ / gsm_F.df_dlambda_
lambda_solved_G = gsm_G.f_ / gsm_G.df_dlambda_

In [ ]:
subs_Sig_Eps_F = dict(zip(gsm_F.Sig.as_explicit(), gsm_F.Sig_.as_explicit()))
subs_Sig_Eps_G = dict(zip(gsm_G.Sig.as_explicit(), gsm_F.Sig_.as_explicit()))
subs_Sig_Eps_F, subs_Sig_Eps_G

In [ ]:
lambda_solved_F.subs(subs_Sig_Eps_F), lambda_solved_G.subs(subs_Sig_Eps_G)

In [ ]:
gsm_F.f_, gsm_G.f_

In [ ]:
gsm_F.df_dSig_,gsm_G.df_dSig_

In [ ]:
gsm_F.Sig_signs

In [ ]:
gsm_F.phi_.diff(gsm_F.Sig.as_explicit())

In [ ]:
gsm_F.Phi_, gsm_G.Phi_

In [ ]:
gsm_G.df_dEps_.T * gsm_G.Phi_

In [ ]:
subs_Sig_Eps_F = dict(zip(gsm_F.Sig.as_explicit(), gsm_F.Sig_.as_explicit()))
delta_Eps_F = sp.simplify((gsm_F.f_/ gsm_F.df_dlambda_)* gsm_F.Phi_).subs(subs_Sig_Eps_F)
subs_Sig_Eps_G = dict(zip(gsm_G.Sig.as_explicit(), gsm_F.Sig_.as_explicit()))
delta_Eps_G = sp.simplify((gsm_G.f_/ gsm_G.df_dlambda_)* gsm_G.Phi_).subs(subs_Sig_Eps_G)
lambda_solved_F, lambda_solved_G

In [ ]:
gsm_F.Phi_, gsm_G.Phi_

In [ ]:
gsm_G.F_expr.diff(gsm_G.u_vars), gsm_G.sig_

In [ ]:
gsm_F.save_to_disk()
gsm_G.save_to_disk()

### Monotonic loading

In [ ]:
_f_c = 44
_f_t = -0.1 * _f_c
_X_0 = (_f_c + _f_t) / 2
_f_s = (_f_c - _f_t) / 2
_E = 50000
_K_factor = 1e-2
_K = _E * _K_factor
material_params = dict(
    E=_E, 
    K=_K / 10, # _E / 5,
    f_c=_f_s,
    eta=20000,
    T_0=20,
    C_v=1e+6, # 0.01, # 0.0001, 
    d_N=1,
    beta=0 # 0.5,
)
p1d.mparams

In [ ]:
def gsm_run(gsm_, u_ta, T_t, t_t, **material_params):
    response = gsm_.get_response(u_ta, T_t, t_t, **material_params)
    _t_t, _u_tIa, _T_t, _Eps_tIb, _Sig_tIb, _iter_t, _dF_dEps_t, lam_t, (d_t_t, d_u_ta) = response
    _u_atI, _Eps_btI, _Sig_btI, _dF_dEps_btI = [np.moveaxis(v_, -1, 0) for v_ in (_u_tIa, _Eps_tIb, _Sig_tIb, _dF_dEps_t)]
    _sig_atI = gsm_.get_sig(_u_atI, _T_t, _Eps_btI, _Sig_btI, **material_params )
    return _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI, _dF_dEps_btI, lam_t, (d_t_t, d_u_ta)

gsm_F.vp_on = True
gsm_F.update_at_k = False

def get_cyclic_load(max_s, max_t, n_cycles, n_incr):
    # Generating loading history
    s_arr = np.tile(np.array([-1, 1]), n_cycles) * np.linspace(0, max_s, 2 * n_cycles)
    s_arr = np.interp(np.linspace(0, max_t, n_incr * len(s_arr)), np.linspace(0, max_t, len(s_arr)), s_arr)

    # time array as input
    t_arr = np.linspace(0, max_t, len(s_arr))
    return s_arr, t_arr

In [ ]:

def generate_cyclic_load(max_s, min_s, freq, total_cycles, points_per_cycle):
    # Calculate the time for one cycle
    total_time = total_cycles / freq

    # Calculate the mean value and amplitude
    mean_value = (max_s + min_s) / 2
    amplitude = (max_s - min_s) / 2

    # Calculate the initial loading slope
    slope = 2 * np.pi * freq * amplitude
    
    # Time arrays for linear increase and sinusoidal part
    initial_duration = mean_value / slope
    initial_points = int(initial_duration * freq * points_per_cycle)
    total_points = int(total_time * freq * points_per_cycle)
    
    # Generate the initial linear increase
    initial_t = np.linspace(0, initial_duration, initial_points, endpoint=False)
    initial_loading = slope * initial_t

    # Generate the sinusoidal loading
    sinusoidal_t = np.linspace(0, total_time, total_points, endpoint=False)
    sinusoidal_loading = mean_value + amplitude * np.sin(2 * np.pi * freq * sinusoidal_t)

    # Combine the initial linear increase with the sinusoidal loading
    t_full = np.concatenate((initial_t, sinusoidal_t + initial_duration))
    s_full = np.concatenate((initial_loading, sinusoidal_loading))
    
    return s_full, t_full

In [ ]:
material_params

In [ ]:
max_s = 0.002
u_t_F, t_t_F = get_cyclic_load( n_cycles=1, max_s=max_s, 
                                max_t=1, n_incr = 3)
# u_t_F, t_t_F = generate_cyclic_load(max_s, -max_s, 1, 10, 30)
T_t_F = 20 + t_t_F * 0
print(u_t_F.shape, T_t_F.shape, t_t_F.shape)
response_values_F = gsm_run(gsm_F, u_t_F[:,np.newaxis], T_t_F, t_t_F, **material_params)

In [ ]:
_t_t_F, _u_atI_F, _sig_atI_F, _T_t_F, _Eps_btI_F, _Sig_btI_F, _dF_dEps_btI_F, lam_t_F, (d_t_t, d_u_ta) = response_values_F 
argmax_sig_ = np.argmax(_sig_atI_F)
t_t_G = _t_t_F # [:argmax_sig_+1]
u_ta_G = _sig_atI_F[...].reshape(-1, 1) # [0, :argmax_sig_+1, 0].reshape(-1, 1)
T_t_G = 20 + t_t_G * 0
response_values_G = gsm_run(gsm_G, u_ta_G, T_t_G, t_t_G, **material_params)

In [ ]:
fig, ((ax_eps, ax_sig), (ax_lam, ax_z), (ax_eps_p, ax_sig_eps)) = plt.subplots(3,2, figsize=(12,10), tight_layout=True)

_t_t_F, _u_atI_F, _sig_atI_F, _T_t_F, _Eps_btI_F, _Sig_btI_F, _dF_dEps_btI_F, _lam_t_F, (_d_t_F, _d_u_ta_F) = response_values_F 
_u_p_atI_F, _z_atI_F = gsm_F.Eps_as_blocks(_Eps_btI_F)
_, _Z_atI_F = gsm_F.Eps_as_blocks(_Sig_btI_F)

_t_t_G, _sig_atI_G, _u_atI_G, _T_t_G, _Eps_btI_G, _Sig_btI_G, _dF_dEps_btI_G, _lam_t_G, (_d_t_G, _d_u_ta_G) = response_values_G 
_u_p_atI_G, _z_atI_G = gsm_G.Eps_as_blocks(_Eps_btI_G)
_, _Z_atI_G = gsm_G.Eps_as_blocks(_Sig_btI_G)

ax_sig_eps.plot(_u_atI_F[0, :, 0], _sig_atI_F[0, :, 0], 'o-', label='F', color='blue',)
ax_sig_eps.plot(_u_atI_G[0, :, 0], _sig_atI_G[0, :, 0], 'o', label='G', color='red', ls='dashed')
ax_sig_eps.legend()
ax_sig_eps.set_title(r'strain-strain')
ax_sig_eps.set_ylabel(r'$\sigma$')
ax_sig_eps.set_xlabel(r'$\varepsilon$')

ax_eps.plot(_t_t_F, _u_atI_F[0, :, 0], 'o-',label='F', color='blue',)
ax_eps.plot(_t_t_G, _u_atI_G[0, :, 0], 'o', label='G', color='red', ls='dashed')
ax_eps.legend()
ax_eps.set_title(r'strain-time')
ax_eps.set_ylabel(r'$\varepsilon$')
ax_eps.set_xlabel(r'$t$')

ax_sig.plot(_t_t_F, _sig_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_sig.plot(_t_t_G, _sig_atI_G[0, :, 0], 'o', label='G', color='red', ls='dashed')
ax_sig.legend()
ax_sig.set_title(r'stress-time')
ax_sig.set_ylabel(r'$\sigma$')
ax_sig.set_xlabel(r'$t$')

ax_lam.plot(_t_t_F, _lam_t_F, 'o-', label='F', color='blue')
ax_lam.plot(_t_t_G, _lam_t_G, 'o-', label='G', color='red')
ax_lam.legend()
ax_lam.set_title(r'$\lambda$-time')
ax_lam.set_ylabel(r'$\lambda$')
ax_lam.set_xlabel(r'$t$')

ax_z.plot(_t_t_F, _z_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_z.plot(_t_t_G, _z_atI_G[0, :, 0], 'o-', label='G', color='red')
ax_z.legend()
ax_z.set_title(r'z-time')
ax_z.set_ylabel(r'$z$')
ax_z.set_xlabel(r'$t$')

ax_eps_p.plot(_t_t_F, _u_p_atI_F[0, :, 0], 'o-', label='F', color='blue')
ax_eps_p.plot(_t_t_G, _u_p_atI_G[0, :, 0], 'o-', label='G', color='red')
ax_eps_p.legend()
ax_eps_p.set_title(r'$\varepsilon_\mathrm{p}$-time')
ax_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$')
ax_eps_p.set_xlabel(r'$t$')


In [ ]:
from bmcs_utils.api import Cymbol

In [ ]:
gsm_F.Eps.as_explicit()

## L_G_EP - Lagrangian manually augmented with consistency condition 

### Mechanical dissipation $\gamma_\mathrm{mech}$

In [ ]:
gamma_mech_ = - (gsm_F.dF_dEps_.as_explicit().T * gsm_F.dot_Eps.as_explicit())[0,0]
gamma_mech_

### Threshold function $f$

In [ ]:
gsm_F.f_

In [ ]:
dot_lam = Cymbol(r'{\lambda}', codename='dot_lambda')

### Lagrangian

In [ ]:
L_ = -gamma_mech_ + dot_lam * gsm_F.f_
L_

In [ ]:
L_Eps_ = L_.subs(gsm_F.subs_Sig_Eps)
L_Eps_

In [ ]:
dL_dEps_ = L_Eps_.diff(gsm_F.Eps.as_explicit())
dL_dEps_

In [ ]:
subs_dot_Eps = sp.solve(dL_dEps_, gsm_F.dot_Eps.as_explicit())
subs_dot_Eps

### Rate of the flow potential $\dot{g}$

In [ ]:
g_Sig_ = gsm_F.phi_

$$
\dot{g} = \frac{\partial g}{\partial \boldsymbol{\mathcal{S}}} \dot{\boldsymbol{\mathcal{S}}}
$$

In [ ]:
dot_g_Sig_ = (g_Sig_.diff(gsm_F.Sig.as_explicit()).T * gsm_F.dot_Sig.as_explicit())[0,0]
dot_g_Sig_

In [ ]:
dot_eps = Cymbol(r'\dot{\varepsilon}', codename='dot_eps')
dot_g_Eps_ = dot_g_Sig_.subs(dict(zip(gsm_F.dot_Sig.as_explicit(), gsm_F.Sig_.as_explicit()))).subs(p1d.eps, dot_eps).subs(dict(zip(gsm_F.Eps.as_explicit(), gsm_F.dot_Eps.as_explicit())))
dot_g_Eps_

In [ ]:
dot_g_Eps_ = dot_g_Eps_.subs(gsm_F.subs_Sig_Eps)
dot_g_Eps_lambda_ = dot_g_Eps_.subs(subs_dot_Eps)
dot_g_Eps_lambda_
dot_g_Eps_, dot_g_Eps_lambda_

In [ ]:
lambda_solved_ = sp.solve(dot_g_Eps_lambda_, dot_lam)[0]
lambda_solved_

In [ ]:
get_lambda_solved = sp.lambdify([dot_eps, p1d.eps] + gsm_F.Eps.as_explicit().tolist() + list(gsm_F.m_params), lambda_solved_, cse=True) 
get_lambda_solved(0.001, 0.1, [0], [0], 20000, 20000, 4, 20000, 0.01, 1)

## R_ep - Lagrangian coupled with consistency condition 

Let us define 
$$
\boldsymbol{\mathcal{A}} = \{ \dot{\boldsymbol{\mathcal{E}}}, \dot{\lambda} \}^T
$$
and
$$
\boldsymbol{\mathcal{R}} = \{ 
    \nabla_{\!\boldsymbol{\mathcal{E}}}\mathcal{L}(\boldsymbol{\mathcal{E}}), 
    \dot{g}(\dot{\boldsymbol{\mathcal{E}}})
    \}^T
$$

### Residuum vector combining Lagrangian and consistency condition

In [ ]:
R_ep_ = dL_dEps_.row_insert(dL_dEps_.shape[0], sp.Matrix([dot_g_Eps_]))
R_ep_

### Vector of unknown internal rates and inelastic multiplier

In [ ]:
A_ep_ = gsm_F.Eps.as_explicit().row_insert(gsm_F.Eps.shape[0], sp.Matrix([[dot_lam]]))
dot_A_ep_ = gsm_F.dot_Eps.as_explicit().row_insert(gsm_F.Eps.shape[0], sp.Matrix([[dot_lam]]))
dot_A_ep_

### Analytically solved rates and multiplier

In [ ]:
dot_A_ep_solved = sp.solve(R_ep_, dot_A_ep_)
dot_A_ep_solved[dot_lam]

### Jacobian of the unkonwn rates

In [ ]:
R_ep_.jacobian(dot_A_ep_)

$$
\boldsymbol{\mathcal{R}}(\varepsilon_{n+1}, \boldsymbol{\mathcal{E}}_{n+1}, \dot{\boldsymbol{\mathcal{A}}}_{n+1}) = 0
$$
$$
{\boldsymbol{\mathcal{E}}}_{n+1} = {\boldsymbol{\mathcal{E}}_n} + \Delta {\boldsymbol{\mathcal{E}}}
$$

 - What would be the Lagrangian that would deliver this system directly as a gradient?
 - Is there a mapping to a dissipation potential (relation to Hackl and Fischer)  

## R_g - Generic Lagrangian augmented with consistency 

### Helmholtz potential and its rate

In [ ]:
class R_g:
    # time
    t = Cymbol(r't', codename='t')
    # state variables
    eps = sp.Function(r'\varepsilon', codename='eps')(t)
    Eps = sp.Function(r'\boldsymbol{\mathcal{E}}', codename='Eps')(t)
    Sig = sp.Function(r'\boldsymbol{\mathcal{S}}', codename='Sig')(eps, Eps)

    # Helmholtz free energy
    psi_ = sp.Function('psi')(eps, Eps)

    # threshold function for elastic domain
    f_ = sp.Function(r'{f}')(Eps, Sig)

    # flow potential
    g_ = sp.Function(r'{g}')(Eps, Sig)

    # time derivatives
    dot_eps_ = eps.diff(t)
    dot_Eps_ = Eps.diff(t)
    dot_Sig_ = Sig.diff(t)
    dot_psi_ = psi_.diff(t)

    # mechanical dissipation
    gamma_mech_ = -psi_.diff(Eps) * dot_Eps_

    # expressions for thermodynamic forces as gradient of the Helmholtz free energy
    Sig_ = psi_.diff(Eps)

    # Lagrangian and its gradient
    dot_lam = Cymbol(r'\dot{\lambda}', codename='dot_lambda')
    L_ = -gamma_mech_ + dot_lam * f_
    dL_dEps_ = L_.diff(Eps)

    # flow potential and its rate
    dot_g_ = g_.diff(t)

    R_ = sp.Matrix([dL_dEps_, dot_g_])



Helmholtz free energy and its rate

In [ ]:
R_g.psi_, R_g.dot_psi_

Mechanical dissipation as a negative rate of change of the Helmholtz free energy with respect to the rate of internal variables

In [ ]:
R_g.gamma_mech_

Thermodynamic foces and their rates as symbols

In [ ]:
R_g.Sig, R_g.dot_Sig_

Dissipation function is the negative Helmholtz rate

Threshold function defined in terms of $\mathcal{S}(\varepsilon, \mathcal{E})$ 

In [ ]:
R_g.f_

### Lagrangian and its gradient

In [ ]:
R_g.L_

In [ ]:
R_g.dL_dEps_

### Consistency criterion - zero rate of flow potential

In [ ]:
R_g.dot_g_

### Residuum function $\mathcal{R}$

In [ ]:
R_g.R_

## R_ig - General incremental Lagrangian

In [ ]:
class R_ig:

    # time
    delta_t = Cymbol(r'\Delta t', codename='delta_t')
    
    # fundamental state
    Eps_n = Cymbol(r'\boldsymbol{\mathcal{E}}_n', codename='Eps_n')
    eps_n = Cymbol(r'\varepsilon_n', codename='eps_n')

    # increment
    delta_Eps_n = Cymbol(r'\Delta{\boldsymbol{\mathcal{E}}}_n', codename='delta_Eps_n')
    delta_eps_n = Cymbol(r'\Delta{\varepsilon}_n', codename='delta_eps_n')
    delta_lam_n = Cymbol(r'\Delta{\lambda}_n', codename='delta_lam_n')

    # updated state
    Eps_n1 = Eps_n + delta_Eps_n
    eps_n1 = eps_n + delta_eps_n

    # rate of change
    dot_Eps_n = delta_Eps_n / delta_t
    dot_eps_n = delta_eps_n / delta_t
    dot_lam_n = delta_lam_n / delta_t

    # derive substitutions
    dot_Eps = R_g.Eps.diff(R_g.t)
    dot_eps = R_g.eps.diff(R_g.t)
    dot_lam = dot_lam

    # substitute the time derivatives
    subs_dot_Eps = {dot_Eps: dot_Eps_n}
    subs_dot_eps = {dot_eps: dot_eps_n}
    subs_dot_lam = {dot_lam: dot_lam_n}
    subs_Eps = {R_g.Eps: Eps_n1}
    subs_eps = {R_g.eps: eps_n1}
    subs_n1 = {**subs_dot_Eps, **subs_dot_eps, **subs_dot_lam, **subs_Eps, **subs_eps}

    # residuum vector in n+1 step
    R_n1 = R_g.R_.subs(subs_n1)

    # construct the jacobian of the residuum
    delta_A_ig_ = sp.Matrix([delta_Eps_n, delta_lam_n])
    dR_dA_n1 = R_n1.jacobian(delta_A_ig_)

R_ig.subs_n1

In [ ]:
R_ig.R_n1

In [ ]:
sp.cse(R_ig.R_n1)

### Construct the Jacobian

In [ ]:
R_ig.delta_A_ig_

In [ ]:
R_ig.dR_dA_n1

In [ ]:
dR_dA_ig_cse_ = sp.cse(R_ig.dR_dA_n1, optimizations='basic')
dR_dA_ig_cse_[0]

In [ ]:
dR_dA_ig_cse_[1][0]

## R_g[.] - Parameterized residuum derivation

Noting that the lagrangian combines not only the values of the internal variables but also their rates
the structure of the solver takes a slightly different form compared to the standard constrained 
optimization problem which is fully defined by introducing the Kuhn-Tucker conditions and constructing 
the Lagrangian. Since the unknowns in our optimization problem are not the internal variables but their rates, the Kuhn-Tucker conditions are not sufficient to solve the problem. Instead, we need to introduce the
rate of the Lagrange multiplier as an additional unknown. The rate of the Lagrange multiplier is
determined by employing the consistency condition ensuring that the rate of the Lagrange multiplier
is such that the constraint is implicitly satisfied. In other words, the rate of the Lagrange multiplier
is determined by the requirement that the constraint is satisfied at the rate level. This is the key
difference between the standard constrained optimization problem and the problem of the rate-based
formulation of the constitutive model.

The generic residuum that we are going to derive 
from the three potentials $\psi$, $f$, and $g$ will have the form:

\begin{align}
\mathcal{R}_{\mathrm{g}} = \left[
\begin{array}{c}
\nabla_{\!\boldsymbol{\mathcal{E}}}\mathcal{L}
(\varepsilon, \boldsymbol{\mathcal{E}}, \dot{\lambda}) \\
\dot{g}(\boldsymbol{\mathcal{E}})
\end{array}
\right]
\end{align}

### Gradient of the Lagrangian

Print the generic form of the Lagrangian gradient $\nabla_{\!\boldsymbol{\mathcal{E}}}\mathcal{L}$ as  the first component of the residuum $\mathcal{R}_\mathrm{g}$. The abstract representation provides a guidance for the construction of the elastoplastic solver $\mathcal{R}_\mathrm{ep}$ starting from the Helmholtz energy $\psi_\mathrm{ep}$ and threshold function $f_\mathrm{ep}$ with the associated flow potential $g_\mathrm{ep} = f_\mathrm{ep}$  

This can rephrased as 
$$
\nabla^2_{\!\boldsymbol{\mathcal{E}}} \psi(\varepsilon, \boldsymbol{\mathcal{E}})
\cdot \dot{\boldsymbol{\mathcal{E}}}
+
\left(
\nabla_{\!\boldsymbol{\mathcal{S}}}f
\cdot
\nabla_{\!\boldsymbol{\mathcal{E}}}\boldsymbol{\mathcal{S}}
+
\nabla_{\!\boldsymbol{\mathcal{E}}}f
\right)
\dot{\lambda} = 0
$$

Note that the threshold function is formulated in terms of thermodynamic forces $\boldsymbol{\mathcal{S}}$ may need to align the sign with the conjugate internal variable. This configurational feature is introduced using the diagnoal matrix $\Upsilon$ containing either values 1 or -1. 
$$
 \boldsymbol{\mathcal{S}} = \boldsymbol{\Upsilon} \nabla_{\!\boldsymbol{\mathcal{E}}} \psi.
$$

### Rate of the flow potential

The general form of the rate of the flow potential is given by the following expression: 
$$
 \dot{g}(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S}}) =
\nabla_{\!\boldsymbol{\mathcal{E}}}g
\cdot
\dot{\boldsymbol{\mathcal{E}}} +
\nabla_{\!\boldsymbol{\mathcal{S}}}g
\cdot
\dot{\boldsymbol{\mathcal{S}}}
$$
where
$$
\dot{\boldsymbol{\mathcal{S}}} = 
\nabla_{\!\varepsilon}\boldsymbol{\mathcal{S}}
\cdot
\dot{\varepsilon} +
\nabla_{\!\boldsymbol{\mathcal{E}}}\boldsymbol{\mathcal{S}}
\cdot
\dot{\boldsymbol{\mathcal{E}}}
$$
so that the whole expression reads
$$
 \dot{g}(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S}}) =
\nabla_{\!\boldsymbol{\mathcal{E}}}g
\cdot
\dot{\boldsymbol{\mathcal{E}}}
+
\nabla_{\!\boldsymbol{\mathcal{S}}}g
\cdot
\left(
\nabla_{\!\varepsilon}\boldsymbol{\mathcal{S}}
\cdot
\dot{\varepsilon}
+
\nabla_{\!\boldsymbol{\mathcal{E}}}\boldsymbol{\mathcal{S}}
\cdot
\dot{\boldsymbol{\mathcal{E}}}
\right)
= 0.
$$

### Combining the two expressions we obtain the whole residuum

Delivering the residuum as
$$
\mathcal{R}_\mathrm{g} = \left[
\begin{array}{c}
\nabla_{\!\boldsymbol{\mathcal{E}}}\mathcal{L} \\
\dot{g}
\end{array}
\right] = \left[
\begin{array}{c}
\nabla^2_{\!\boldsymbol{\mathcal{E}}} \psi
\cdot \dot{\boldsymbol{\mathcal{E}}} +
\left(
\nabla_{\!\boldsymbol{\mathcal{S}}}f
\cdot
\nabla_{\!\boldsymbol{\mathcal{E}}}\boldsymbol{\mathcal{S}} +
\nabla_{\!\boldsymbol{\mathcal{E}}}f
\right)
\dot{\lambda} \\
\nabla_{\!\boldsymbol{\mathcal{E}}}g
\cdot
\dot{\boldsymbol{\mathcal{E}}} +
\nabla_{\!\boldsymbol{\mathcal{S}}}g
\cdot
\left(
\nabla_{\!\varepsilon}\boldsymbol{\mathcal{S}}
\cdot
\dot{\varepsilon} +
\nabla_{\!\boldsymbol{\mathcal{E}}}\boldsymbol{\mathcal{S}}
\cdot
\dot{\boldsymbol{\mathcal{E}}}
\right)
\end{array}
\right] =
\boldsymbol{0}
$$

In [ ]:
def derive_R_g(psi_, eps, dot_eps, Eps, dot_Eps, Sig, Sig_signs, f_, g_, dot_lam):
    """
    Derive the residuum vector combining the gradient of the Lagrangian and the rate of the flow potential.

    Parameters
    ----------
    psi_ : sympy expression
        The free energy potential as a function of internal variables.
    eps : sympy symbol
        The external variable.
    dot_eps : sympy symbol
        The rate of change of the external variable.
    Eps : sympy matrix
        The vector of internal variables.
    dot_Eps : sympy matrix
        The rate of change of the internal variables.
    Sig : sympy matrix
        The conjugate vector of thermodynamic forces.
    Sig_signs : list or array-like
        Array of signs.
    f_ : sympy expression
        The yield function.
    g_ : sympy expression
        The flow potential function.
    dot_lam : sympy symbol
        The rate of change of the plastic multiplier.

    Returns
    -------
    sympy matrix
        The residuum vector combining the gradient of the Lagrangian and the rate of the flow potential.

    Notes
    -----
    The derivation involves computing the gradient of the Lagrangian with respect to internal variables
    and the rate of change of the flow potential.
    """
    # Compute the gradient of the Lagrangian
    Sig_ = sp.simplify(sp.diag(*Sig_signs) * psi_.diff(Eps))
    dL_dEps_ = (sp.hessian(psi_, Eps).T * dot_Eps + 
        ((f_.diff(Sig).T * Sig_.jacobian(Eps)).T + f_.diff(Eps)) * dot_lam ).subs(dict(zip(Sig, Sig_)))

    # Compute the rate of the flow potential
    dot_psi_eps_ = Sig_.diff(eps) * dot_eps
    dot_psi_Eps_ = Sig_.jacobian(Eps) * dot_Eps
    dot_Sig_Eps_ = dot_psi_eps_ + dot_psi_Eps_
    dot_g_ = g_.diff(Eps).T * dot_Eps + (g_.diff(Sig).T * dot_Sig_Eps_).subs(dict(zip(Sig, Sig_)))

    # Combine the gradient of the Lagrangian and the rate of the flow potential
    R_g_ = dL_dEps_.row_insert(dL_dEps_.shape[0], sp.Matrix([dot_g_]))

    return R_g_

### Example 1: R_g[ep] - Isotropic hardening plasticity

The specification of the constitutive model is done in terms of three functions. Here we exemplify the isotropic hardening plasticity in terms of 
$$
\psi_\mathrm{ep} := \frac{1}{2} E (\varepsilon - \varepsilon_\mathrm{p})^2 + \frac{1}{2} K z^2
$$ 
and the yield function
$$
 f_\mathrm{ep} := \left| {\sigma_\mathrm{p}} \right| - (f_\mathrm{c} + Z)
$$

In [ ]:
class P_ep:
    # external variables, internal variables, thermodynamic forces, and material parameters
    eps, eps_p, z = sp.symbols(r'\varepsilon \varepsilon_p z', real=True)
    sig, sig_p, Z = sp.symbols('sigma sigma_p Z', real=True)
    E, K, f_c = sp.symbols('E K f_c', positive=True)
    # free energy potential, yield function, and flow potential
    psi_ = sp.Rational(1, 2) * E * (eps - eps_p)**2 + sp.Rational(1, 2) * K * z**2
    f_ = sp.Abs(sig_p) - (f_c + Z)
    g_ = f_
    # collect the internal variables to enable generic derivation of the residuum
    Eps = sp.Matrix([eps_p, z])
    Sig = sp.Matrix([sig_p, Z])
    Sig_signs = [-1, 1]
    mparams = (E, K, f_c)
    # general functions - to be factored out to a base class?
    dot_Eps = sp.Matrix([sp.Symbol(name=f'\\dot{{{var.name}}}') for var in list(Eps)])
    dot_eps = sp.Symbol(name=f'\\dot{{{eps.name}}}')

# Access the attributes directly from the class
P_ep.psi_, P_ep.f_, P_ep.Eps, P_ep.dot_Eps, P_ep.Sig, P_ep.Sig_signs, P_ep.mparams


In [ ]:
R_gep_ = derive_R_g(P_ep.psi_, P_ep.eps, P_ep.dot_eps, P_ep.Eps, P_ep.dot_Eps, P_ep.Sig, P_ep.Sig_signs, P_ep.f_, P_ep.g_, dot_lam)
R_gep_, R_ep_

#### Check the analytical solution

For isotropic hardening, this equation system can be solved explicitly - use it to check the symbolic scheme.

In [ ]:
dot_A_gep_ = P_ep.dot_Eps.row_insert(P_ep.dot_Eps.shape[0], sp.Matrix([[dot_lam]]))
sp.simplify(sp.solve(R_gep_, dot_A_gep_))

### Example 2: R_g[epd] Isotropic hardening plasticity and damage

The specification of the constitutive model is done in terms of three functions. Here we exemplify the isotropic hardening plasticity in terms of 
$$
\psi_\mathrm{} := \frac{1}{2} (1 - \omega) E (\varepsilon - \varepsilon_\mathrm{p})^2 + \frac{1}{2} K z^2,
$$ 
the yield function
$$
 f_\mathrm{epd} := \left| \frac{\sigma_\mathrm{p}}{1 - \omega} \right| - (f_\mathrm{c} + Z),
$$
and the flow potential
$$
 g_\mathrm{epd} = f + (1 - \omega) \frac{S}{r+1} \left( \frac{Y}{S} \right)^{r+1}
$$

In [ ]:
class P_epd:
    # external variables, internal variables, thermodynamic forces, and material parameters
    eps, eps_p, z, omega = sp.symbols(r'\varepsilon \varepsilon_p z \omega', real=True)
    sig, sig_p, Z, Y = sp.symbols('sigma sigma_p Z Y', real=True)
    E, K, f_c, S, r = sp.symbols('E K f_c S r', positive=True)
    # free energy potential, yield function, and flow potential
    psi_ = sp.Rational(1,2) * (1 - omega) * E * (eps - eps_p)**2 + sp.Rational(1,2) * K * z**2
    f_ = sp.Abs(sig_p / (1-omega)) - (f_c + Z)
    g_ = f_ + (1 - omega) * (S/(r+1)) * (Y/S)**(r+1)
    # collect the internal variables to enable generic derivation of the residuum
    Eps = sp.Matrix([eps_p, z, omega])
    Sig = sp.Matrix([sig_p, Z, Y])
    Sig_signs = [-1, 1, -1]
    mparams = (E, K, f_c, S, r)
    # general functions - to be factored out to a base class?
    dot_Eps = sp.Matrix([sp.Symbol(name=f'\\dot{{{var.name}}}') for var in list(Eps)])
    dot_eps = sp.Symbol(name=f'\\dot{{{eps.name}}}')
    
P_epd.psi_, P_epd.f_, P_epd.Eps, P_epd.dot_Eps, P_epd.Sig, P_epd.Sig_signs, P_epd.mparams

In [ ]:
R_gepd_ = derive_R_g(P_epd.psi_, P_epd.eps, P_epd.dot_eps, P_epd.Eps, P_epd.dot_Eps, P_epd.Sig, P_epd.Sig_signs, P_epd.f_, P_epd.g_, dot_lam)
sp.cse(R_gepd_)

## R_ig[.] - Parameterized incremental Residuum

In [ ]:
def derive_R_dR_n1(P):
    # time
    t = Cymbol(r't', codename='t')
    delta_t = Cymbol(r'\Delta t', codename='delta_t')
    dot_lam = Cymbol(r'\dot{\lambda}', codename='dot_lambda')
    
    # fundamental state
    Eps_n = sp.Matrix([sp.Symbol(f'{var.name}^(n)') for var in P.Eps])
    eps_n = Cymbol(r'\varepsilon_n', codename='eps_n')

    # increment
    delta_Eps_n = sp.Matrix([sp.Symbol(f'\\Delta{{{var.name}}}^{{(n)}}') for var in P.Eps])
    delta_eps_n = Cymbol(r'\Delta{\varepsilon}_n', codename='delta_eps_n')
    delta_lam_n = Cymbol(r'\Delta{\lambda}_n', codename='delta_lam_n')

    # updated state
    Eps_n1 = Eps_n + delta_Eps_n
    eps_n1 = eps_n + delta_eps_n

    # rate of change
    dot_Eps_n = delta_Eps_n / delta_t
    dot_eps_n = delta_eps_n / delta_t
    dot_lam_n = delta_lam_n / delta_t

    # derive substitutions
    dot_Eps = P.Eps.diff(t)
    dot_eps = P.eps.diff(t)

    subs_dot_Eps = dict(zip(P.dot_Eps, dot_Eps_n))
    subs_dot_eps = {dot_eps: dot_eps_n}
    subs_dot_lam = {dot_lam: dot_lam_n}
    subs_Eps = dict(zip(P.Eps, Eps_n1))
    subs_eps = {P.eps: eps_n1}

    subs_n1 = {**subs_dot_Eps, **subs_dot_eps, **subs_dot_lam, **subs_Eps, **subs_eps}

    R_g = derive_R_g(P.psi_, P.eps, P.dot_eps, P.Eps, P.dot_Eps, P.Sig, P.Sig_signs, P.f_, P.g_, dot_lam)
    # residuum vector in n+1 step
    R_n1 = R_g.subs(subs_n1)

    # construct the jacobian of the residuum
    delta_A_ig_ = sp.Matrix([delta_Eps_n, delta_lam_n])
    dR_dA_n1_ = R_n1.jacobian(delta_A_ig_)
    dR_dA_n1 = dR_dA_n1_.replace(sp.Derivative, lambda *args: 0)

    return R_n1, dR_dA_n1


Construct the residuum and its jacobian for elasto plasticity with isotropic hardening

In [ ]:
R_ep_, dR_dA_ep_  = derive_R_dR_n1(P_ep)
R_ep_, dR_dA_ep_

Construct an elasto-plasticity with isotropic hardening model and damage 

In [ ]:
R_epd_, dR_dA_epd_  = derive_R_dR_n1(P_epd)
sp.cse([R_epd_, dR_dA_epd_])

## Next step identification

$$
\boldsymbol{\mathcal{R}}(\Delta{\boldsymbol{\mathcal{A}}}) = 0
$$

$$
\boldsymbol{\mathcal{R}}(\Delta{\boldsymbol{\mathcal{A}}}_{k+1}) =
\boldsymbol{\mathcal{R}}(\Delta{\boldsymbol{\mathcal{A}}}_k) +
\left.
    \frac{\partial \boldsymbol{\mathcal{R}} 
}{\partial \Delta{\boldsymbol{\mathcal{A}}}} 
\right|_{\Delta{\boldsymbol{\mathcal{A}}}_k}
\delta(\Delta{\boldsymbol{\mathcal{A}}}) = 0
$$

\begin{align}
\left.
    \frac{\partial \boldsymbol{\mathcal{R}} 
}{\partial \Delta{\boldsymbol{\mathcal{A}}}} 
\right|_{\Delta{\boldsymbol{\mathcal{A}}}_k}
\delta (\Delta{\boldsymbol{\mathcal{A}}}) &= - 
\boldsymbol{\mathcal{R}}(\Delta{\boldsymbol{\mathcal{A}}}_k) \\
\Delta{\boldsymbol{\mathcal{A}}}_{k+1} &= 
\Delta{\boldsymbol{\mathcal{A}}}_k + 
\delta (\Delta{\boldsymbol{\mathcal{A}}})
\end{align}


In [ ]:
def get_state_n1(eps_n, d_eps, d_t, Sig_n, Eps_n, k_max, **kw):
    """
    Calculates the state at time n+1 based on the given inputs using an iterative algorithm.

    Args:
        eps_n: Strain at time n.
        d_eps_n1: Strain increment from time n to n+1.
        dt: Time step size.
        Sig_n: Stress at time n.
        Eps_n: Strain at time n.
        k_max: Maximum number of iterations.
        **kw: Additional keyword arguments.

    Returns:
        Tuple containing the updated strain Eps_k, stress Sig_k, temperature T_n+1, number of iterations k, 
        and dissipation rate gradient dDiss_dEps.
    """
    eps_n1 = eps_n + d_eps

    Eps_k = np.copy(Eps_n)
    Sig_k = np.copy(Sig_n)
    #### Here for Gibbs - different values for Eps_k and Sig_k - when viscosity on - WHY?
    # print('u_n1', u_n1, T_n, Eps_k, Sig_k)
    f_k, R_k, dR_k, Sig_k = get_R_dR_n1(eps_n, d_eps, d_t, Eps_k, Sig_k, **kw)
    # if f_k > 0:
    #     print('===================== u_n1', u_n1, T_n, Eps_k, Sig_k)
    f_k = np.atleast_1d(f_k)
    df_k = np.atleast_1d(df_k)
    f_k_norm = np.fabs(f_k)
    f_k_trial = np.copy(f_k)
    lam_k = np.zeros_like(f_k)
    k = 0
    while k < k_max:
        I_ = np.logical_and(f_k_trial > 0, f_k_norm >= 1e-4)
        if np.all(I_ == False):
            break # convergence reached
        I = np.where(I_)
        bI = (slice(None), *I)
        lam_k[I] -= f_k[I] / df_k[I] # increment of lambda with delta_lambda = -f / df
        Eps_k[bI] = self.get_Eps_k1(u_n1[bI], T_n[I], Eps_n[bI], lam_k[I], Eps_k[bI], Sig_k[bI], **kw)

        f_k[I], df_k[I], Sig_k[bI] = self.get_f_df_Sig(u_n1[bI], T_n[I], Eps_k[bI], Sig_k[bI], **kw)

        if np.any(np.isnan(f_k[I])):
            print('there is nan in f_k')
            raise RuntimeError(f'there is nan in f_k {I}')

        if np.any(np.isnan(Eps_k[bI])):
            print('there is nan in Eps_k')
            raise RuntimeError(f'there is nan in Eps_k {I}')

        f_k_norm[I] = np.fabs(f_k[I])
        k += 1
    else:
        raise RuntimeError(f'no convergence for indexes {I}')

    dEps_k = Eps_k - Eps_n
    dDiss_dEps = self.get_dDiss_dEps(u_n1, T_n, Eps_k, Sig_k, **kw)
    # dissipation rate
    dDiss_dt = np.einsum('b...,b...->...', dDiss_dEps, dEps_k)
    C_v_ = kw['C_v_']
    d_T = 0 # d_T_n + d_t * (dDiss_dt / C_v_ )# / rho_'

    return np.moveaxis(Eps_k, 0, -1), np.moveaxis(Sig_k, 0, -1), T_n + d_T, k, np.moveaxis(dDiss_dEps, 0, -1), lam_k


## Brief review of inequality treatment using Lagrangian multipliers

Constrained optimization problems involve finding the maximum or minimum of an objective function subject to certain constraints. When dealing with inequality constraints, a common approach involves the use of the Lagrangian formulation combined with the Karush-Kuhn-Tucker (KKT) conditions.

### Lagrangian for Inequality Constraints

Suppose we have an optimization problem:

- Objective function: $ f(x) $
- Inequality constraints: $ g_i(x) \leq 0 $ for $ i = 1, \ldots, m $

The Lagrangian for this problem is constructed as:

$$ \mathcal{L}(x, \lambda) = f(x) + \sum_{i=1}^{m} \lambda_i g_i(x) $$

where $ \lambda_i $ are the Lagrange multipliers associated with each inequality constraint $ g_i(x) \leq 0 $.

### KKT Conditions

The solution to the constrained optimization problem satisfies the following KKT conditions:

1. **Stationarity**: 
   $$ \nabla f(x^*) + \sum_{i=1}^{m} \lambda_i^* \nabla g_i(x^*) = 0 $$

2. **Primal Feasibility**: 
   $$ g_i(x^*) \leq 0 \quad \forall i $$

3. **Dual Feasibility**: 
   $$ \lambda_i^* \geq 0 \quad \forall i $$

4. **Complementary Slackness**: 
   $$ \lambda_i^* g_i(x^*) = 0 \quad \forall i $$

### Application in Elasto-Plasticity with Isotropic Hardening

In the context of elasto-plasticity, the yield function $ f(x, H) \leq 0 $ defines the boundary between elastic and plastic behavior, where $ x $ represents the stress state, and $ H $ represents the hardening variable that evolves with plastic deformation.

#### Consistency Condition

The consistency condition requires that a system remains on the yield surface during plastic deformation, which implies that the rate of change of the yield function $ \dot{f}(x, H) = 0 $.

In this scenario:
- The rate of change should respect the yield surface condition $\dot{f}(x, H) = \frac{\partial f}{\partial x} \dot{x} + \frac{\partial f}{\partial H} \dot{H} = 0$.
- The Lagrange multiplier (plastic multiplier) $ \dot{\lambda} $ is determined such that this condition holds, often leading to a system of equations solving for stress updates and plastic strain increments.

### Conclusion

The use of the Lagrangian and the corresponding KKT conditions in elasto-plasticity allows us to systematically handle the constraints imposed by material laws, ensuring that the plastic flow remains consistent with the constitutive model. The consistency condition is crucial in determining the evolution of plastic variables, ensuring accurate simulation of material behavior under loading.

Understanding how to determine Lagrange multipliers in constrained optimization problems is fundamental, yet it's often surrounded by some confusion. Let's walk through the general approach.

### Lagrangian and KKT Conditions

In general constrained optimization with an objective function $ f(x) $ and constraints $ g_i(x) \leq 0 $, as well as possible equality constraints $ h_j(x) = 0 $, the Lagrangian is constructed as:

$$ \mathcal{L}(x, \lambda, \nu) = f(x) + \sum_{i=1}^m \lambda_i g_i(x) + \sum_{j=1}^p \nu_j h_j(x) $$

where $ \lambda_i $ are the Lagrange multipliers for inequality constraints and $ \nu_j $ for equality constraints.

### Determining the Lagrange Multipliers

1. **Stationarity Condition**:

   The stationarity condition is key and states that at the optimal point, the gradient of the Lagrangian with respect to the primal variables must be zero:

   $$
   \nabla_x \mathcal{L}(x^*, \lambda^*, \nu^*) = \nabla f(x^*) + \sum_{i=1}^m \lambda_i^* \nabla g_i(x^*) + \sum_{j=1}^p \nu_j^* \nabla h_j(x^*) = 0
   $$

   This equation helps determine the relationship between $ x $, $ \lambda $, and $ \nu $. It ensures that the gradients of the constraints and the objective function are balanced by the Lagrange multipliers.

2. **Complementary Slackness**:

   For inequality constraints, complementary slackness provides another crucial condition:

   $$
   \lambda_i^* g_i(x^*) = 0 \quad \forall i
   $$

   This means that if $ g_i(x^*) < 0 $, then $ \lambda_i^* = 0 $; if $ g_i(x^*) = 0 $ (active constraint), $ \lambda_i^* $ can be positive and needs to be determined from other conditions.

3. **Feasibility Conditions**:

   These include both primal and dual feasibility, ensuring $ g_i(x^*) \leq 0 $, $ h_j(x^*) = 0 $, and $ \lambda_i^* \geq 0 $.

4. **Additional Equations**:

   In practice, solving these conditions involves substituting known values and systematically solving the system of equations implied by these conditions. The equations often have to be solved simultaneously, as they are interdependent.

### Solution Process

The process typically involves:

- Solving the KKT conditions as a system of nonlinear equations.
- For simple problems, this can be done analytically (solving the equations directly).
- For more complex problems, numerical methods such as interior-point methods, sequential quadratic programming, or other iterative algorithms are used to approximate the solution.

### Conclusion

In essence, you don't need an additional criterion like the consistency condition used in plasticity modeling, because the KKT conditions inherent to the Lagrangian framework already provide the necessary structure to solve for both the optimal solution and the Lagrange multipliers. The challenge lies in accurately solving these conditions, often requiring a combination of analytical insights and computational techniques.

## Deferred differentiation of a function 

### Define the symbols and abstract functions

We start by defining the symbols and abstract functions that we will use in our derivation.
$t$ is a symbol representing time.
$\varepsilon$ is a function of $t$ representing strain.
$\sigma$ is a function of $\varepsilon$ representing stress.
$f$ is a function of $\sigma$ representing some function of stress.

In [ ]:
t = sp.symbols(r't')
eps = sp.Function(r'\varepsilon')(t)
sig = sp.Function('\sigma')(eps)
f = sp.Function('f')(sig)
t, eps, sig, f

### Define the derivative of `f` with respect to `t`

We want to find the derivative of `f` with respect to time `t`.
Since `f` is a function of `sig`, and `sig` is a function of `eps`, and `eps` is a function of `t`,
we use the chain rule to find the derivative.

In [ ]:
df_deps_dt = f.diff(t)
df_deps_dt

### Define a specific function for `f`

Now, let's define a specific function for `f`. For example, let's take `f` to be `sig^2`.

In [ ]:
specific_f = sig**2
specific_f

### Substitute the specific function into the derivative

We substitute our specific function `f = sig^2` into the derivative we found earlier.


In [ ]:
specific_df_dt = df_deps_dt.subs(f, specific_f)
specific_df_dt

### Express `sig` as a function of `eps`

To evaluate the derivative, we need to express `sig` as a function of `eps`.
For example, let's take `sig = eps^2`.


In [ ]:

sig_expr = eps**2

# ## Substitute `sig` into the expression
# 
# We substitute `sig = eps^2` into the derivative expression.

specific_sig_df_dt = specific_df_dt.subs(sig, sig_expr)
specific_sig_df_dt

# ## Express `eps` as a function of `t`
# 
# Finally, we need to express `eps` as a function of `t`.
# For example, let's take `eps = t^2`.

eps_expr = t**2

# ## Substitute `eps` into the expression and evaluate
# 
# We substitute `eps = t^2` into the derivative expression and evaluate it.

specific_eps_df_dt = specific_sig_df_dt.subs(eps, eps_expr)
specific_eps_df_dt.doit()

In [ ]:
# Define the symbols and abstract functions
t = sp.symbols(r't')  # $t$
eps = sp.Function(r'\varepsilon')(t)  # $\varepsilon(t)$
sig = sp.Function('\sigma')(eps)  # $\sigma(\varepsilon)$
f = sp.Function('f')(sig)  # $f(\sigma)$

# Define the derivative of f with respect to t
df_deps_dt = f.diff(t)  # $\frac{d f}{d t}$

# Define a specific function for f, say f = sig^2
specific_f = sig**2  # $f = \sigma^2$

# Substitute the specific function into the derivative
specific_df_dt = df_deps_dt.subs(f, specific_f)  # $\frac{d (\sigma^2)}{d t}$

# To evaluate it, you also need to express sig as a function of eps, say sig = eps^2
sig_expr = eps**2  # $\sigma = \varepsilon^2$

# Substitute sig into the expression
specific_sig_df_dt = specific_df_dt.subs(sig, sig_expr)  # $\frac{d (\varepsilon^4)}{d t}$

# Express eps as a function of t, say eps = t^2
eps_expr = t**2  # $\varepsilon = t^2$

# Substitute eps into the expression and evaluate
specific_eps_df_dt = specific_sig_df_dt.subs(eps, eps_expr)  # $\frac{d (t^8)}{d t}$
specific_eps_df_dt.doit()  # Evaluate the derivative